In [1]:
from  transformers  import XLMRobertaModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained('xlm-roberta-base')
print('Model and Tokenizer Loaded.')

Model and Tokenizer Loaded.


In [ ]:
import torch
import csv 

def load_word_senses(input_file, word,   model , tokenizer): 
    dico = {}
    with open(input_file, 'r' , encoding='utf8') as f: 
        csv_reader = csv.reader(f , delimiter='\t')
        for row in csv_reader:
            if len(row) < 2:
                print('Skipping Line:' , row)
                continue
                
            tokens = tokenizer(row[1])
            lst = tokenizer.convert_ids_to_tokens(tokens['input_ids']) 

            #Pass it to Model 
            input_ids = torch.tensor(tokens['input_ids']).unsqueeze(0)  
            if input_ids.shape[0] > 500 or input_ids.shape[1] > 500:
                print('Too long line - Skipping:')
                continue 
                
            outputs = model(input_ids)
            #Build the Vectors
            for ind , item in enumerate(lst): 
                if ind >= outputs[0].shape[1]:
                    break
                if item == '<s>' or item == '</s>':
                    continue 
                else: 
                    item = item.replace('_' , '') 
                    if item == word: 
                        print ('({}) {}'.format(row[0], row[1]))
                        if  row[0] not in dico:
                            dico[row[0]] = []  
                        dico[row[0]].append(outputs[0][0, ind , : ]) 

    return dico  

milk_items= load_word_senses('../data/common/milk.txt', 'شیر',   model , tokenizer)
    

In [ ]:
import torch

def  collect_vectors(corpus_file, words , model , tokenizer , max_found = 200): 
    dico = {}
    with open(corpus_file, 'r' , encoding='utf8') as f: 
        found = 0 
        for i, line in enumerate(f):
            
            if line.strip() == '' :
                continue
            l = line.strip().replace('.' , ' ')
            in_list = False 
            added_line = ' ' + l + ' '
            for w in words: 
                if ' ' + w + ' ' in added_line:
                    in_list = True 
                    break 
            if not in_list:
                continue 
                
            tokens = tokenizer(l)
            lst = tokenizer.convert_ids_to_tokens(tokens['input_ids']) 
            #do not pass it to model if none of the words appear in the list
            in_list = False
            for ind , item in enumerate(lst): 
                if item == '<s>' or item == '</s>':
                    continue 
                else: 
                    item = item.replace('_' , '') 
                    if item in words: 
                        in_list = True 
                        break 
            if not in_list:
                continue 
            #Pass it to Model 
            input_ids = torch.tensor(tokens['input_ids']).unsqueeze(0)  
            if input_ids.shape[0] > 500 or input_ids.shape[1] > 500:
                print('Too long line - Skipping:')
                continue 
                
            outputs = model(input_ids)
            #Build the Vectors
            for ind , item in enumerate(lst): 
                if ind >= outputs[0].shape[1]:
                    break
                if item == '<s>' or item == '</s>':
                    continue 
                else: 
                    item = item.replace('_' , '') 
                    if item in words: 
                        found += 1
                        print ('({}) {}'.format(found, l))
                        if  item not in dico:
                            dico[item] = []  
                        dico[item].append(outputs[0][0, ind , : ]) 
            if found > max_found:
                break
    return dico  


all_vectors = collect_vectors('../data/common/wiki.txt' , ['شیر'] , model, tokenizer)

#print (tokenizer('یلام لبلنل لانمیب انیلب'))


In [ ]:
from sklearn.decomposition import PCA
import numpy as np 
import matplotlib.pyplot as plt

def draw_in_2d(data_dictionary):
    pca = PCA(n_components=2)
    total_vectors = 0
    dim = 0 

    for key, value in data_dictionary.items(): 
        print('---' , key , ' ' , len(value))

        total_vectors += len(value)
        if len(value) > 0:
            dim = value[0].shape[0]

    x = np.zeros((total_vectors, dim))
    rev_index = {} 
    ind = 0 
    for key, value in data_dictionary.items(): 
        print('--' , key)
        rev_index[key] = [] 
        for vec in value:
            rev_index[key].append(ind) 
            print(key)
            x[ind , :] = vec.detach().numpy() 
            ind += 1

    x_2d = pca.fit_transform(x)
    print(rev_index)
    for key in data_dictionary: 
        plt.scatter(x_2d[ rev_index[key] , 0] , x_2d[ rev_index[key] , 1] , label=key , color=np.random.rand(3,))
    plt.show()

draw_in_2d(milk_items)